# importe de modelo

In [ ]:
from tensorflow.keras.applications import MobileNetV2

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense,Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import image as image_ops
import matplotlib.pyplot as plt
import numpy as np
import os



In [ ]:
img_width,img_height=224,224

model=MobileNetV2(weights='imagenet',
                include_top=False,
                input_shape=(img_height,img_width,3)
                )

In [ ]:
def add_layer_at_bottom(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

# Aqui se cargan los datos

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
directorio_training='D:/chestData2/train'
val_data_dir='D:/chestData2/test'
os.listdir(directorio_training)


In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                
                                 
                                 horizontal_flip=True,
                                 fill_mode='nearest')

val_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
batch_size=32

In [ ]:
entrenador=train_datagen.flow_from_directory(directorio_training,
                                                  target_size=(img_height,img_width),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

validador=val_datagen.flow_from_directory(val_data_dir,
                                              target_size=(img_height,img_width),
                                              batch_size=batch_size,
                                              class_mode='categorical')
test_images,test_labels= next(validador)


In [ ]:
L = 4
W = 4
imagen_entrenamiento,labels_entrenamiento= next(entrenador)
fig, axes = plt.subplots(L, W, figsize = (12, 12))
axes = axes.ravel()
label_names = {0 : 'Covid-19', 1 : 'Normal' , 2: 'Viral Pneumonia'}

for i in np.arange(0, L*W):
    axes[i].imshow(imagen_entrenamiento[i])
    axes[i].set_title(label_names[np.argmax(labels_entrenamiento[i])])
    
    plt.subplots_adjust(wspace = 0.5)   

In [ ]:
train_class_names = set()
num_train_samples=0
for i in entrenador.filenames:
    train_class_names.add(i.split('/')[0])
    num_train_samples+=1
#print(num_train_samples)
#train_class_names

In [ ]:
val_class_names = set()
num_val_samples=0
for i in validador.filenames:
    val_class_names.add(i.split('/')[0])
    num_val_samples+=1
print(num_val_samples)
#val_class_names

In [ ]:
num_classes=len(entrenador.class_indices)
print(num_classes)
FC_head=add_layer_at_bottom(model,
                            num_classes)

main_model=Model(inputs=model.input,
                 outputs=FC_head)

main_model.summary()

# Training

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:

checkpoint = ModelCheckpoint("enfermedad_dec.h6",
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=20,
                          verbose=1,
                          restore_best_weights=True)

callbacks=[checkpoint,earlystop]

main_model.compile(loss='categorical_crossentropy',
                   optimizer=RMSprop(learning_rate=0.0015),
                   metrics=['accuracy'])

epochs=50

batch_size = 32

history = main_model.fit(entrenador,
                         steps_per_epoch=num_train_samples//batch_size,
                         epochs=epochs,
                         callbacks=callbacks,
                         validation_data=validador,
                         validation_steps=num_val_samples//batch_size)



In [ ]:
# plot accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
# import tkinter as tk
# from tkinter import filedialog
# import cv2
# import numpy as np
# from matplotlib import pyplot as plt

# # Aquí se debería importar y cargar el modelo entrenado 'main_model'
# def abrir_archivo():
#     file = filedialog.askopenfilename(initialdir="/", title="Seleccione archivo")
#     if file:
#         print("Archivo seleccionado: ", file)
#         out=['Covid', 'Normal', 'Viral Pneumonia']
#         img=cv2.imread(file)
#         img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#         img=cv2.resize(img,(224,224))
#         img=img/255.
#         mostrar_imagen_con_titulo(img, out)

# def mostrar_imagen_con_titulo(img, out):
#     plt.imshow(img)
#     img=img.reshape(1,224,224,3)
#     res=main_model.predict(img)
#     print(res)
#     plt.title(out[np.argmax(res)])
#     plt.show()

# app = tk.Tk()
# app.geometry("400x300")
# app.title("Selector de archivos")

# button = tk.Button(app, text="Abrir archivo", command=abrir_archivo)
# button.pack(pady=20)

# app.mainloop()

In [ ]:
# import tkinter as tk
# from tkinter import filedialog
# import os
# import cv2
# import random
# import string
# import matplotlib.pyplot as plt
# import numpy as np

# out = ['Covid', 'Normal', 'Viral Pneumonia']

# def cambiar_nombres_carpeta_rnd(carpeta_ruta):
#     lista_archivos = os.listdir(carpeta_ruta)
#     print(carpeta_ruta)

#     for nombre_archivo in lista_archivos:
#         ruta_completa_antigua = os.path.join(carpeta_ruta, nombre_archivo)

#         if os.path.isfile(ruta_completa_antigua):
#             extension = os.path.splitext(nombre_archivo)[1]
#             nuevo_nombre = ''.join(random.choices(string.ascii_lowercase, k=10)) + extension
#             ruta_completa_nueva = os.path.join(carpeta_ruta, nuevo_nombre)

#             try:
#                 os.rename(ruta_completa_antigua, ruta_completa_nueva)
#                 print(f"Archivo {nombre_archivo} renombrado a {nuevo_nombre}")
#             except Exception as e:
#                 print(f"No se pudo cambiar el nombre del archivo {nombre_archivo}. Error: {e}")

# def seleccionar_carpeta_y_clasificar():
#     carpeta_ruta = filedialog.askdirectory()
#     etiqueta_ruta.config(text=f"Ruta de la carpeta seleccionada: {carpeta_ruta}")

#     for imagen_nombre in os.listdir(carpeta_ruta):
#         if imagen_nombre.endswith(('.jpg', '.jpeg', '.png')):
#             imagen_ruta = os.path.join(carpeta_ruta, imagen_nombre)
#             img = cv2.imread(imagen_ruta)
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#             img = cv2.resize(img, (224, 224))
#             img = img / 255.
#             plt.imshow(img)
#             img = img.reshape(1, 224, 224, 3)
#             # Aquí deberías tener definido el modelo preentrenado como main_model
#             res = main_model.predict(img)
#             print(res)
#             print(out[np.argmax(res)])
            
            
            
#             plt.title(out[np.argmax(res)])
#             plt.show()

# # Configuración de la ventana
# ventana = tk.Tk()
# ventana.title("Selector de Carpeta e Impresión de Imágenes")

# # Botón para seleccionar la carpeta y clasificar imágenes
# boton_seleccionar = tk.Button(ventana, text="Seleccionar Carpeta e Imprimir Imágenes", command=seleccionar_carpeta_y_clasificar)
# boton_seleccionar.pack(pady=20)

# # Etiqueta para mostrar la ruta seleccionada
# etiqueta_ruta = tk.Label(ventana, text="Ruta de la carpeta seleccionada: ")
# etiqueta_ruta.pack(pady=10)

# # Ejecutar la ventana
# ventana.mainloop()


In [ ]:
# import tkinter as tk
# from tkinter import filedialog
# import os
# import cv2
# import numpy as np
# from tensorflow.keras.models import load_model
# import matplotlib.pyplot as plt



# # Función para obtener y mostrar las imágenes
# def mostrar_imagenes_carpeta():
#     carpeta_ruta = filedialog.askdirectory()
#     etiqueta_ruta.config(text=f"Ruta de la carpeta seleccionada: {carpeta_ruta}")

#     # Obtener las imágenes y mostrarlas
#     fig = plt.figure(figsize=(10, 10))

#     for i, file in enumerate(os.listdir(carpeta_ruta)[:9]):
#         if file.endswith(".jpeg") or file.endswith(".jpg") or file.endswith(".png"):
#             img_path = os.path.join(carpeta_ruta, file)
#             img = cv2.imread(img_path)
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#             img = cv2.resize(img, (224, 224))
#             img = img / 255.
#             img = img.reshape(1, 224, 224, 3)
#             res = main_model.predict(img)
#             prediction = out[np.argmax(res)]

#             ax = fig.add_subplot(3, 3, i + 1)
#             ax.imshow(img[0])
#             ax.set_title(f'Prediction: {prediction}')
#             ax.axis('off')

#     plt.show()

# # Configuración de la ventana
# ventana = tk.Tk()
# ventana.title("Selector de Carpeta y Visualizador de Imágenes")

# # Botón para seleccionar la carpeta y mostrar imágenes
# boton_seleccionar = tk.Button(ventana, text="Seleccionar Carpeta y Mostrar Imágenes", command=mostrar_imagenes_carpeta)
# boton_seleccionar.pack(pady=20)

# # Etiqueta para mostrar la ruta seleccionada
# etiqueta_ruta = tk.Label(ventana, text="Ruta de la carpeta seleccionada: ")
# etiqueta_ruta.pack(pady=10)

# # Ejecutar la ventana
# ventana.mainloop()


In [ ]:
import tkinter as tk
from tkinter import filedialog
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image, ImageTk

# Carga el modelo (asegúrate de reemplazar 'ruta_al_modelo' con la ruta real del modelo)
#main_model = load_model('ruta_al_modelo')
out=['Covid', 'Normal', 'Viral Pneumonia']

# Configuración de la ventana de tkinter
ventana = tk.Tk()
ventana.title("Selector de Carpeta y Visualizador de Imágenes")
ventana.geometry("800x800")

# Función para obtener y mostrar las imágenes
def mostrar_imagenes_carpeta():
    carpeta_ruta = filedialog.askdirectory()
    etiqueta_ruta.config(text=f"Ruta de la carpeta seleccionada: {carpeta_ruta}")

    row_counter = 1
    column_counter = 0

    for file in os.listdir(carpeta_ruta)[:9]:
        if file.endswith(".jpeg") or file.endswith(".jpg") or file.endswith(".png"):
            img_path = os.path.join(carpeta_ruta, file)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))
            
            
            imgPre = img / 255.
            imgPre = imgPre.reshape(1, 224, 224, 3)
            res = main_model.predict(imgPre)
            prediction = out[np.argmax(res)]
            
            
            img = Image.fromarray(img)
            img = ImageTk.PhotoImage(img)
            panel = tk.Label(ventana, image=img)
            panel.image = img
            panel.grid(row=row_counter, column=column_counter, padx=10, pady=10)

            

            etiqueta_prediccion = tk.Label(ventana, text=f"Prediction: {prediction}")
            etiqueta_prediccion.grid(row=row_counter + 1, column=column_counter, padx=10)

            column_counter += 1
            if column_counter > 2:
                column_counter = 0
                row_counter += 2

# Etiqueta para mostrar la ruta seleccionada
etiqueta_ruta = tk.Label(ventana, text="Ruta de la carpeta seleccionada: ")
etiqueta_ruta.grid(row=0, column=0, padx=10, pady=10)

# Botón para seleccionar la carpeta y mostrar imágenes
boton_seleccionar = tk.Button(ventana, text="Seleccionar Carpeta y Mostrar Imágenes", command=mostrar_imagenes_carpeta)
boton_seleccionar.grid(row=1, column=0, padx=10, pady=10)

# Ejecutar la ventana
ventana.mainloop()
